# Data Analysis

In [ ]:
import pandas as pd

df = pd.read_json('responses.json', lines=True)

## Sentiment Analysis with Twitter-roBERTa-base for Sentiment Analysis with and without fine-tuning

In [ ]:
import os
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, pipeline
from datasets import load_dataset
from torch.utils.data import DataLoader

sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

def analyze_sentiment(text):
    result = sentiment_pipeline(text)
    print(result)
    return result

df['sentiment'] = df['response'].apply(lambda x: analyze_sentiment(x))

In [ ]:
dataset = load_dataset("classla/parlasent")
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

if not os.path.exists("./fine_tuned_roberta_parlasent"):
    # Definire funzione di tokenizzazione
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    # Applicare la tokenizzazione al dataset
    tokenized_datasets = dataset.map(tokenize_function, batched=True)

    # Preparare il dataset per il training
    tokenized_datasets = tokenized_datasets.remove_columns(["text"])
    tokenized_datasets.set_format("torch")

    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=10,
        push_to_hub=False,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
    )

    trainer.train()
    trainer.save_model("./fine_tuned_roberta_parlasent")


In [ ]:
from torch import softmax

def analyze_sentiment_finetuned(text: str) -> float:
    # Tokenizza il testo
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    
    # Calcola le previsioni
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    
    # Calcola la probabilità usando softmax
    probs = softmax(logits, dim=-1).squeeze().cpu().numpy()    
    # Trova la classe con la probabilità massima
    sentiment_class = probs.argmax()
    
    return 2 * (sentiment_class - 1)  # -1, 0, 1 corrispondenti a negative, neutral, positive

df['sentiment_score_finetuned'] = df['response'].apply(lambda x: analyze_sentiment_finetuned(x))

## Entity recognition with BERT-base-ner

In [ ]:
ner_pipeline = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple")

def mentions_china_pipeline(text):
    entities = ner_pipeline(text)
    for entity in entities:
        if entity["entity_group"] == "LOC" and entity["word"].lower() in {"china", "beijing", "cpc", "chinese communist party"}:
            return True
    return False


df['prompt_about_china'] = df['prompt'].apply(lambda x: mentions_china_pipeline(x))
df['response_about_china'] = df['response'].apply(lambda x: mentions_china_pipeline(x))

## Entity recognition with dbmdz/bert-large-cased-finetuned-conll03-english

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

model_name = "dbmdz/bert-large-cased-finetuned-conll03-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

china_related_keywords = {
    "China", "Chinese", "Beijing", "CPC", "Communist Party of China",
    "Huawei", "Alibaba", "Belt and Road Initiative", "Xi Jinping"
}

def mentions_china_pipeline_finetuned(text):
    """ Esegue Named Entity Recognition (NER) e filtra le entità correlate alla Cina """
    entities = ner_pipeline(text)
    for word in entities:
        if word['word'] in china_related_keywords:
            return True
    return False


df['prompt_about_china_finetuned'] = df['prompt'].apply(lambda x: mentions_china_pipeline_finetuned(x))
df['response_about_china_finetuned'] = df['response'].apply(lambda x: mentions_china_pipeline_finetuned(x))


## Pre-Trained stance classification with RoBERTa

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch.nn.functional as F

# Carica il modello e il tokenizer
MODEL_NAME = "cardiffnlp/twitter-roberta-base-stance"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

# Label originali del modello
LABELS = ["against", "neutral", "favor"]

def get_stance_label(scores):
    """Restituisce la label con il punteggio più alto."""
    return LABELS[torch.argmax(scores)]

def classify_stance(text):
    """Classifica il testo come Pro Communism, Neutral o Pro Capitalism, restituendo anche la probabilità."""
    topics = ["communism", "capitalism"]
    stance_results = {}

    for topic in topics:
        inputs = tokenizer(f"{text} [SEP] {topic}", return_tensors="pt")
        outputs = model(**inputs)
        scores = F.softmax(outputs.logits, dim=1).squeeze()
        label = get_stance_label(scores)
        probability = scores[torch.argmax(scores)].item()  # Estrai la probabilità della classe scelta
        stance_results[topic] = (label, probability)

    # Mappatura ai nostri label con probabilità
    if stance_results["communism"][0] == "favor" and stance_results["capitalism"][0] == "against":
        return "Pro Communism", stance_results["communism"][1]
    elif stance_results["capitalism"][0] == "favor" and stance_results["communism"][0] == "against":
        return "Pro Capitalism", stance_results["capitalism"][1]
    elif stance_results["communism"][0] == "against" and stance_results["capitalism"][0] == "against":
        return "Against both", (stance_results["communism"][1], stance_results["capitalism"][1])
    elif stance_results["communism"][0] == "favor" and stance_results["capitalism"][0] == "favor":
        return "Pro both", (stance_results["communism"][1], stance_results["capitalism"][1])
    else:
        return "Neutral", (stance_results["communism"][1], stance_results["capitalism"][1])


df['stance'] = df['response'].apply(lambda x: classify_stance(x))


## saving results

In [ ]:
if os.path.exists('processed_results.json'):
    if os.path.exists('processed_results_old.json'):
        os.remove('processed_results_old.json')
    os.rename('processed_results.json', 'processed_results_old.json')
df.to_json('processed_results.json', orient='records')